In [ ]:
from core.utils import Tibanna
from core import ff_utils
import copy

env = 'fourfront-webprod'
tibanna = Tibanna(env=env)
ff = ff_utils.fdn_connection(key=tibanna.ff_keys)

def sort_p_files(p_file_list, acc):
    archive = []
    new_p_files = copy.deepcopy(p_file_list)
    deleted = []
    
    for p_file in p_file_list:
        p_file_resp = ff_utils.get_metadata(p_file, connection = ff, frame= 'embedded')
        
        # get rid of all deleted files
        if p_file_resp['status'] == 'deleted':
            deleted.append(p_file)
            continue

        p_wfr =  p_file_resp['workflow_run_outputs']

        assert len(p_wfr) == 1
        wfr_name = p_wfr[0]['display_title']

        if '0.2.5' not in wfr_name:
            archive.append(p_file)
            new_p_files.remove(p_file)
    
    rep = 'no change'
    if archive:       
        rep = 'proc_files will be archived'
        if new_p_files:
            rep += ' partially'
        if deleted:
            rep += ', cleaned deleted items'
    elif deleted:
        rep = 'cleaned deleted items'
    
    
        # add archived
        patch_data_1 = {'archived_processed_files': archive}
        try:
            ff_utils.patch_metadata(patch_data_1, obj_id=acc ,connection=ff)
        except Exception as e:
            print e 
        
        # update processed files
        patch_data_2 = {'processed_files': new_p_files}
        try:
            ff_utils.patch_metadata(patch_data_2, obj_id=acc ,connection=ff)
        except Exception as e:
            print e 
    return archive, new_p_files, rep

In [ ]:
# work on all experiment sets

url = '/search/?status=released&status=released%20to%20project&status=archived&status=submission%20in%20progress&type=ExperimentSetReplicate'
sets = ff_utils.get_metadata(url, connection=ff)['@graph']
print len(sets), 'experiment sets'
pros_sets = [i for i in sets if i.get('processed_files')]
print len(pros_sets), 'sets with processed files'
stati = [i['status'] for i in sets if i.get('processed_files')]
print 'with statuses', ",".join(list(set(stati))).upper()

In [ ]:
# work on experiment set part 2
for a_set in pros_sets:
    exps = a_set['experiments_in_set']
    p_files = a_set['processed_files']
    # take care of sets first
    arch, new_p, rep = sort_p_files(p_files, a_set['accession'])
    print 'set', a_set['accession'], rep
    
    for exp in exps:
        p_files_exp = ff_utils.get_metadata(exp, connection = ff)
        arch_e, new_p_e, rep_e = sort_p_files(p_files_exp['processed_files'], p_files_exp['accession'])
        print 'exp', p_files_exp['accession'], rep_e
    print

In [ ]:
# Work on individual types of experiments

exp_type = 'ExperimentHiC'
url_hic = '/search/?status=released&status=released%20to%20project&status=archived&status=submission%20in%20progress&type='+exp_type
hic_exps = ff_utils.get_metadata(url_hic, connection=ff)['@graph']
print len(hic_exps), 'experiment hic'

pros_hic = [i for i in hic_exps if i.get('processed_files')]
print len(pros_hic), 'hics with processed files'

stati_hic = [i['status'] for i in hic_exps if i.get('processed_files')]
print 'with statuses', ",".join(list(set(stati_hic))).upper()

for a_hic in pros_hic:
    p_files = a_hic['processed_files']
    # take care of sets first
    arch, new_p, rep = sort_p_files(p_files, a_hic['accession'])
    print 'hic', a_hic['accession'], rep